In [1]:
from pathlib import Path
import sys, os


FOLDERNAME = Path.home() / "ece562" / "assignment1_colab" / "assignment1"

assert FOLDERNAME.exists(), f"[!] Folder not found: {FOLDERNAME}"

sys.path.append(str(FOLDERNAME))

os.chdir(FOLDERNAME)
print("CWD:", os.getcwd())

CWD: /Users/maxim/ece562/assignment1_colab/assignment1


# Pruning

After learning, neural networks have modified and learned a set of parameters to perform our classification task. However, such parameters are costly to maintain and do not hold the same importance.

Wouldn't it be great could optimize our resource usage by dropping less important values ? This is where pruning comes into play.

Pruning is a technique that cuts off parameters/structures from a model to increase sparcity and decrease overall model size, similar to cutting leafs or branches from bushes and trees. This process can lead to smaller memory consumption with minimal accuracy reduction. Moreover, pruning the network may also provide a speedup since there will be less operations being performed.

The pruning process can be performed during the end of an epoch of training or after training is complete. Experimenting to find out which way works the best is part of the fun !

In [17]:
from ece697ls.pruning_helper import test_model, load_model #, get_trained_model 
from ece697ls.data_utils import get_CINIC10_data
import torch.nn.utils.prune as prune
import torch
import os

Below we will load a pre-trained model for you to work on. If you prefer, you can save your own model from the previous Tensorflow/Pytorch task and load it here.

In [59]:
#This code may take a while to execute as it is training a network form scratch

data = get_CINIC10_data()
mode = 'torch'#torch or tensorflow

test_data = [data['X_test'],data['y_test']]

path = os.path.join(f"ece697ls/models/{mode}.model")

model = load_model(path,mode=mode)
test_model(model,test_data,mode=mode)

Test Acc: 0.5138


## Unstructured Pruning

Unstructured Pruning is usually related to the pruning of weights in neural networks. The general idea is to select a set of weights according to a policy and setting them up to zero. 

Common policies are random weight selection or selecting the smallers weights. 
Unstructured Pruning can be performed in one or multiple layers within the same network.

Altough in theory Unstructured Pruning should decrease the number of operations performed during execution there should be explicit support within the framework or hardware to bypass such operations, otherwise it will just operated over zero.

### Perform Pruning

Using the model trained in the previous step using pytorch, perform unstructured pruning in the weights of the model by removing x% of the smallest weights. 

*   Increment global pruning by 10% until reaching total of 80% pruned weights
*   Perform inference at the end of each pruning and observe the impact into the accuracy.


Note: The percentages are related to the entire model, not per layer.



In [58]:
################################################################################
# TODO: Perform unstructured Pruning over the trained model using 3 different  
# prunning percentages.                                
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model.eval()

#Show the accuracy before any pruning
test_model(model, test_data, mode='torch')

modules_to_prune = []
for name, module in model.named_modules():
    if isinstance(module, (torch.nn.Linear, torch.nn.Conv2d)):
        modules_to_prune.append((module, 'weight'))

percentages = [0.4, 0.5, 0.6, 0.8] 

print("Start")

for sparsity in percentages:
    print("\nPruning to " + str(sparsity * 100))

    prune.global_unstructured(modules_to_prune, pruning_method=prune.L1Unstructured, amount=sparsity)
    test_model(model, test_data, mode='torch')

print("End")
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

Test Acc: 0.5212
Start

Pruning to 40.0
Test Acc: 0.5108

Pruning to 50.0
Test Acc: 0.4848

Pruning to 60.0
Test Acc: 0.3868

Pruning to 80.0
Test Acc: 0.1963
End


## Inline Question 1:

What happened with the accuracy as the % of pruning increased ?
Why was that the case?


## Answer: 

[FILL YOU ANSWER HERE]

## Structured Pruning

Structured Pruning consists of removing a bigger chunk of the network parameters at the same time. Instead of removing only a few weights, it is commonplace to remove entire neurons. 

For example, in Convolutional Layers, removing filters can be beneficial to improve performance as it greatly decreases the amount of computation performed. However, some of these changes may affect output dimensions which may be carried over to other parts of the network. Therefore, when performing structured pruning one must always be aware of which parameters are going to be affected.

Using the previously trained model in the CINIC-10, perform Structured Prunning only in the Convolution layers of the DNN.

In [60]:
################################################################################
# TODO: Perform unstrucuted Pruning over the trained model using 3 different  
# prunning percentages.                                
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model.eval()

test_model(model, test_data, mode='torch')

percentages = [0.4, 0.5, 0.6, 0.8]

print("Start")
for sparsity in percentages:
    model = load_model(path, num_classes=num_classes, mode='torch') 
    model.eval() 
    
    print("\nPruning to " + str(sparsity * 100))
    prune.ln_structured(model.conv1, name='weight', amount=sparsity, n=1,dim=0) 
    prune.ln_structured(model.conv2, name='weight',amount=sparsity,n=1, dim=0)

    test_model(model, test_data, mode='torch')

print("End")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

Test Acc: 0.5212
Start

Pruning to 40.0
Test Acc: 0.1729

Pruning to 50.0
Test Acc: 0.1855

Pruning to 60.0
Test Acc: 0.2282

Pruning to 80.0
Test Acc: 0.1989
End


## Inline Question 2:

What is the difference between performing Structured Pruning vs Dropout ? 
Why would it be beneficial to perform both techniques when developing a Neural Network?


## Answer: 

[FILL YOU ANSWER HERE]
